### ETL S3 servidores públicos

In [35]:
import json
import csv
from pprint import pprint
from typing import Dict, Any, Tuple, List, Optional

In [36]:
### ruta de los archivos json
ruta_datos_entrada = '../pruebas/datos_entrada'
ruta_datos_salida = '../pruebas/datos_salida'

# Apuntando a un solo archivo de entrada para pruebas en s3s
#""" archivo_entrada_s3s = '/s3s/MICHOACAN/data-00001.json' """
#archivo_entrada_s3s = '/s3s/CHIAPAS/data-0000000001.json'
archivo_entrada_s3s = '/s3s/QUERETARO/data-0000000001.json'
#nameFiles = glob.glob('*.json')

In [44]:
def extract_faltas_element(element_data: Dict[str, Any]) -> Tuple[Dict[str, Any], List[str], List[str]]:
    def safe_get(obj: Dict[str, Any], *attrs):
        for attr in attrs:
            if isinstance(obj, dict) and attr in obj:
                obj = obj[attr]
            else:
                return None
        return obj
    # Inicializar listas de campos presentes y faltantes en el elemento json
    campos_presentes = []
    campos_faltantes = ["empleoCargoComision/nivelOrdenGobierno",]

    # Construir el diccionario de datos generales
    datos_generales = {}
    empleoCargoComision = {}
    faltaCometida = {}
    resolucion = {}
    tipoSancion = {}

    # Mapeo de campos
    mappings = {
        "fecha": ("fechaCaptura",),
        "expediente": ("expediente",),
        "datosGenerales/nombres": ("servidorPublicoSancionado", "nombres"),
        "datosGenerales/primerApellido": ("servidorPublicoSancionado", "primerApellido"),
        "datosGenerales/segundoApellido": ("servidorPublicoSancionado", "segundoApellido"),
        "datosGenerales/curp": ("servidorPublicoSancionado", "curp"),
        "datosGenerales/rfc": ("servidorPublicoSancionado", "rfc"),
        "datosGenerales/sexo": ("servidorPublicoSancionado", "genero", "valor"),
        "empleoCargoComision/entidadFederativa": ("institucionDependencia", "nombre"),
        #"empleoCargoComision/nivelOrdenGobierno":{},
        "empleoCargoComision/nombreEntePublico": ("institucionDependencia", "nombre"),
        "empleoCargoComision/siglasEntePublico": ("institucionDependencia", "siglas"),
        "empleoCargoComision/nivelJerarquico/clave": ("servidorPublicoSancionado", "nivel"),
        "empleoCargoComision/nivelJerarquico/valor": ("servidorPublicoSancionado", "nivel"),
        "empleoCargoComision/denominacion": ("servidorPublicoSancionado", "puesto"),
        "faltaCometida/clave": ("tipoFalta", "clave"),
        "faltaCometida/valor": ("tipoFalta", "valor"),
        "faltaCometida/descripcionHechos": ("causaMotivoHechos",),
        "resolucion/fechaResolucion": ("resolucion", "fechaResolucion"),
        "resolucion/autoridadResolutora": ("autoridadSancionadora",),
        "tipoSancion/clave": ("tipoSancion", 0, "clave"),
        "tipoSancion/sancionEconomica/monto": ("multa", "monto"),
        "tipoSancion/sancionEconomica/moneda": ("multa", "moneda", "valor"),
        "tipoSancion/inhabilitacion/plazoAnios": ("inhabilitacion", "plazo"),
        "tipoSancion/inhabilitacion/fechaInicial": ("inhabilitacion", "fechaInicial"),
        "tipoSancion/inhabilitacion/fechaFinal": ("inhabilitacion", "fechaFinal"),
        "observaciones": ("observaciones",)
    }

    processed_element = {}

    for key, path in mappings.items():
        value = safe_get(element_data, *path)
        if value is not None:
            if key.startswith("datosGenerales"):
                datos_generales[key[14:]] = value
            elif key.startswith("empleoCargoComision"):
                empleoCargoComision[key[19:]] = value
            elif key.startswith("faltaCometida"):
                faltaCometida[key[13:]] = value
            elif key.startswith("resolucion"):
                resolucion[key[10:]] = value
            elif key.startswith("tipoSancion"):
                tipoSancion[key[11:]] = value
            else:
                processed_element[key] = value
            campos_presentes.append(key)
        else:
            campos_faltantes.append(key)

    if datos_generales:
        processed_element["datosGenerales"] = datos_generales
    if empleoCargoComision:
        processed_element["empleoCargoComision"] = empleoCargoComision
    if faltaCometida:
        processed_element["faltaCometida"] = [faltaCometida]
    if resolucion:
        processed_element["resolucion"] = resolucion
    if tipoSancion:
        processed_element["tipoSancion"] = [tipoSancion]

    return processed_element, campos_presentes, campos_faltantes


In [45]:
# Leer el JSON desde el archivo
objeto_json = ruta_datos_entrada + archivo_entrada_s3s
with open(objeto_json, 'r', encoding='utf-8') as f:
    datos_leidos = json.load(f)


In [46]:
pprint(datos_leidos)

[{'autoridadSancionadora': 'SALA ESPECIALIZADA EN MATERIA DE RESPONSABILIDADES '
                           'ADMINISTRATIVAS DEL TRIBUNAL DE JUSTICIA '
                           'ADMINISTRATIVA DEL ESTADO DE QUERETARO',
  'causaMotivoHechos': 'EN FECHA 05 DE MAYO DE 2021, SE RECIBIO EN LA '
                       'DIRECCION DE INVESTIGACION DE LA SECRETARIA DE LA '
                       'CONTRALORIA DEL PODER EJECUTIVO DEL ESTADO DE '
                       'QUERETARO, LA PROMOCION CIUDADANA 2021W2488 FORMULADA '
                       'POR VANESSA GUTIERREZ MONROY, EN LA QUE DENUNCIO: QUE '
                       'LA DENUNCIADA, MA. TERESA ADRIANA FLORES RUIZ, '
                       'PRESIDENTE AUXILIAR DE LA JUNTA ESPECIAL UNO, TRAMITO '
                       'EL CUMPLIMIENTO DE LA SENTENCIA DE AMPARO 788/2020 DEL '
                       'JUZGADO PRIMERO DISTRITO DE PROCESOS PENALES FEDERALES '
                       'Y DE AMPARO DE QUERETARO, EN LA QUE EL ABOGADO DE UNA '
    

In [47]:
# Pasar los datos leídos a la función extract_faltas_element
resultado, campos_presentes, campos_faltantes = extract_faltas_element(datos_leidos[0])

# Imprimir los resultados
print("Datos reestructurados:")
print(json.dumps(resultado, ensure_ascii=False, indent=2))


Datos reestructurados:
{
  "fecha": "2023-09-21T00:00:00Z",
  "expediente": "2301002-SG-SE",
  "observaciones": "EXISTEN ELEMENTOS DE PRUEBA QUE ACREDITAN QUE LA PRESUNTA RESPONSABLE NO SE EXCUSO DE CONOCER Y TRAMITAR EL JUICIO LABORAL 974/2012/1/1 Y EL INCIDENTE 788/202-II, YA QUE EN DICHO PROCEDIMIENTO EL ABOGADO DE LA PARTE ACTORA ES SU HERMANO, POR ELLO, QUEDO ACREDITADO QUE LA ENCAUSADA NO SE EXCUSO PARA ACTUAR EN EL JUICIO LABORAL 974/2012/1/1 Y EL INCIDENTE 788/202-II, DONDE EL ABOGADO DE LA PARTE ACTORA ES SU HERMANO Y POR ELLO ACTUO BAJO CONFLICTO DE INTERES, LO CUAL CONDUCE A DETERMINAR QUE MEDIANTE SU CONDUCTA CONTRAVINO EL PRINCIPIO DE DISCIPLINA DE IMPARCIALIDAD PREVISTO EN EL ARTICULO 7 FRACCION IX, DE LA LEY GENERAL DE RESPONSABILIDADES ADMINISTRATIVAS, POR LO CUAL SE HACE CREEDORA A LA SANCION DE INHABILITACION POR UN AÑO QUE CORRESPONDE AL MAXIMO DEL MARGEN CUANDO NO EXISTA BENEFICIO O LUCRO ALGUNO. SE TIENE EN CUENTA COMO HECHO NOTORIO QUE EN FECHA 11 DE MARZO DE 2022

In [48]:
print("\nCampos presentes:", campos_presentes)
print("\nCampos faltantes:", campos_faltantes)

# Si deseas guardar el resultado reestructurado en un nuevo archivo JSON
with open('datos_reestructurados.json', 'w', encoding='utf-8') as f:
    json.dump(resultado, f, ensure_ascii=False, indent=2)

print("\nDatos reestructurados guardados en 'datos_reestructurados.json'")


Campos presentes: ['fecha', 'expediente', 'datosGenerales/nombres', 'datosGenerales/primerApellido', 'datosGenerales/segundoApellido', 'empleoCargoComision/entidadFederativa', 'empleoCargoComision/nombreEntePublico', 'empleoCargoComision/denominacion', 'faltaCometida/clave', 'faltaCometida/valor', 'faltaCometida/descripcionHechos', 'resolucion/fechaResolucion', 'resolucion/autoridadResolutora', 'tipoSancion/inhabilitacion/plazoAnios', 'tipoSancion/inhabilitacion/fechaInicial', 'tipoSancion/inhabilitacion/fechaFinal', 'observaciones']

Campos faltantes: ['empleoCargoComision/nivelOrdenGobierno', 'datosGenerales/curp', 'datosGenerales/rfc', 'datosGenerales/sexo', 'empleoCargoComision/siglasEntePublico', 'empleoCargoComision/nivelJerarquico/clave', 'empleoCargoComision/nivelJerarquico/valor', 'tipoSancion/clave', 'tipoSancion/sancionEconomica/monto', 'tipoSancion/sancionEconomica/moneda']

Datos reestructurados guardados en 'datos_reestructurados.json'


In [49]:
def extraer_nombre_archivo(ruta: str) -> str:
    return ruta.split('/')[-1].split('.')[0]

# Ejemplo de uso
ruta = '/s3s/QUERETARO/data-0000000001.json'
nombre_archivo = extraer_nombre_archivo(archivo_entrada_s3s)
reporte_faltantes = nombre_archivo + "_faltantes.txt"


In [50]:
with open(reporte_faltantes, "a", newline="") as faltantes_file:
    writer = csv.writer(faltantes_file)
    writer.writerow(campos_faltantes)

### Separar objetos por faltas graves y no graves

In [39]:
### importar bibliotecas de python para realizar la migracion
import json
from addict import Dict
import glob
from operator import attrgetter
from pprint import pprint
from typing import Dict, Any, Tuple, List, Optional

In [41]:
### varibables globales
array_faltas_servidores_graves = []
array_faltas_servidores_no_graves = []


In [42]:
### diccionario de faltas graves
tipo_de_faltas_graves = ["ABUSO_FUNCIONES", "COHECHO", "PECULADO", "DESVIO_RECURSOS_PUBLICOS", "UTILIZACION_INDEBIDA_INFORMACION", "CONFLICTO_INTERES", "CONTRATACION_INDEBIDA", "ENRIQUECIMIENTO_OCULTO", "TRAFICO_INFLUENCIAS", "SIMULACION_ACTO_JURIDICO", "ENCUBRIMIENTO", "DESACATO", "NEPOTISMO", "OBSTRUCCION"]

In [43]:
### diccionario de faltas no graves
tipo_de_faltas_no_grave = ["CUMPLIR_FUNCIONES", "DENUNCIAR_FALTAS", "ATENDER_INSTRUCCIONES", "DECLARACIONES", "CUSTODIAR_DOCUMENTACION", "SUPERVISAR_ART_49_LGRA", "RENDIR_CUENTAS", "COLABORAR_PROCEDIMIENTOS","CAUSAR_DANOS"]

In [44]:
### Se carga los datos de los archivos JSON en un array de objetos Dict
def load_json_files(file_pattern):
    all_data = []
    for filename in glob.glob(file_pattern):
        with open(filename, 'r', encoding='utf-8') as file:
            json_data = json.load(file)
            # Convertir cada objeto JSON a un objeto Dict de addict
            all_data.extend([Dict(item) for item in json_data])
    return all_data

In [45]:
### Cargar todos los archivos JSON que se ubiquen en la ruta especificada
data = load_json_files(ruta_datos_entrada + archivo_entrada_s3s)

In [37]:
pprint(data)

[{'autoridadSancionadora': 'DATOS NO PROPORCIONADOS',
  'causaMotivoHechos': 'DATOS NO PROPORCIONADOS',
  'expediente': 'CA/012/2022',
  'fechaCaptura': '2022-05-06T00:00:00Z',
  'id': 'b411f02b-5ea7-489e-93e5-9eca24d090f6',
  'inhabilitacion': {'fechaFinal': '2022-05-15',
                     'fechaInicial': '2022-02-15',
                     'plazo': '3 MESES'},
  'institucionDependencia': {'clave': '78',
                             'nombre': 'Fiscalia General del Estado',
                             'siglas': 'FGE'},
  'multa': {'moneda': {'clave': 'MXN', 'valor': 'PESO MEXICANO'}, 'monto': 0},
  'resolucion': {'fechaResolucion': '2022-01-12',
                 'url': 'DATOS NO PROPORCIONADOS'},
  'servidorPublicoSancionado': {'nivel': 'DATOS NO PROPORCIONADOS',
                                'nombres': 'ANDRES',
                                'primerApellido': 'REYES',
                                'puesto': 'FISCAL DEL MINISTERIO PUBLICO, '
                                   

In [76]:
def extract_faltas_element(element_data: Dict[str, Any]) -> Tuple[Dict[str, Any], List[str], List[str]]:
    def safe_get(obj: Dict[str, Any], *attrs):
        for attr in attrs:
            if isinstance(obj, dict) and attr in obj:
                obj = obj[attr]
            else:
                return None
        return obj

    campos_presentes = []
    campos_faltantes = []

    # Construir el diccionario de datos generales
    datos_generales = {}
    empleoCargoComision = {}
    faltaCometida = {}
    resolucion = {}
    tipoSancion = {}

    # Mapeo de campos
    mappings = {
        "fecha": ("fechaCaptura",),
        "expediente": ("expediente",),
        "datosGeneralesnombres": ("servidorPublicoSancionado", "nombres"),
        "datosGeneralesprimerApellido": ("servidorPublicoSancionado", "primerApellido"),
        "datosGeneralessegundoApellido": ("servidorPublicoSancionado", "segundoApellido"),
        "datosGeneralescurp": ("servidorPublicoSancionado", "curp"),
        "datosGeneralesrfc": ("servidorPublicoSancionado", "rfc"),
        "datosGeneralessexo": ("servidorPublicoSancionado", "genero", "valor"),
        "empleoCargoComisionentidadFederativa": ("institucionDependencia", "nombre"),
        "empleoCargoComisionnombreEntePublico": ("institucionDependencia", "nombre"),
        "empleoCargoComisionsiglasEntePublico": ("institucionDependencia", "siglas"),
        "empleoCargoComision/nivelJerarquicoclave": ("servidorPublicoSancionado", "nivel"),
        "empleoCargoComision/nivelJerarquicovalor": ("servidorPublicoSancionado", "nivel"),
        "empleoCargoComisiondenominacion": ("servidorPublicoSancionado", "puesto"),
        "faltaCometida[]clave": ("tipoFalta", "clave"),
        "faltaCometida[]valor": ("tipoFalta", "valor"),
        "faltaCometida[]descripcionHechos": ("causaMotivoHechos",),
        "resolucionfechaResolucion": ("resolucion", "fechaResolucion"),
        "resolucionautoridadResolutora": ("autoridadSancionadora",),
        "tipoSancion[]clave": ("tipoSancion", 0, "clave"),
        "tipoSancion[]/sancionEconomicamonto": ("multa", "monto"),
        "tipoSancion[]/sancionEconomicamoneda": ("multa", "moneda", "valor"),
        "tipoSancion[]/inhabilitacionplazoAnios": ("inhabilitacion", "plazo"),
        "tipoSancion[]/inhabilitacionfechaInicial": ("inhabilitacion", "fechaInicial"),
        "tipoSancion[]/inhabilitacionfechaFinal": ("inhabilitacion", "fechaFinal"),
        "observaciones": ("observaciones",)
    }

    processed_element = {}

    for key, path in mappings.items():
        value = safe_get(element_data, *path)
        if value is not None:
            if key.startswith("datosGenerales"):
                datos_generales[key[14:]] = value
            elif key.startswith("empleoCargoComision"):
                empleoCargoComision[key[19:]] = value
            elif key.startswith("faltaCometida"):
                faltaCometida[key[13:]] = value
            elif key.startswith("resolucion"):
                resolucion[key[10:]] = value
            elif key.startswith("tipoSancion"):
                tipoSancion[key[11:]] = value
            else:
                processed_element[key] = value
            campos_presentes.append(key)
        else:
            campos_faltantes.append(key)

    if datos_generales:
        processed_element["datosGenerales"] = datos_generales
    if empleoCargoComision:
        processed_element["empleoCargoComision"] = empleoCargoComision
    if faltaCometida:
        processed_element["faltaCometida"] = [faltaCometida]
    if resolucion:
        processed_element["resolucion"] = resolucion
    if tipoSancion:
        processed_element["tipoSancion"] = [tipoSancion]

    return processed_element, campos_presentes, campos_faltantes

# Ejemplo de uso
json_data =   {
    "id": "251",
    "fechaCaptura": "2023-09-21T00:00:00Z",
    "expediente": "2301002-SG-SE",
    "institucionDependencia": {
    "nombre": "Secretaría del Trabajo"
    },
    "servidorPublicoSancionado": {
    "nombres": "MA. TERESA ADRIANA",
    "primerApellido": "FLORES",
    "segundoApellido": "RUIZ",
    "puesto": "PRESIDENTA AUXILIAR DE LA JUNTA ESPECIAL NUMERO UNO DE LA LOCAL DE CONCILIACION Y ARBITRAJE DE LA SECRETARIA DEL TRABAJO DEL PODER EJECUTIVO DEL ESTADO DE QUERETARO"
    },
    "autoridadSancionadora": "SALA ESPECIALIZADA EN MATERIA DE RESPONSABILIDADES ADMINISTRATIVAS DEL TRIBUNAL DE JUSTICIA ADMINISTRATIVA DEL ESTADO DE QUERETARO",
    "tipoFalta": {
    "clave": "ABCI",
    "valor": "ACTUACION BAJO CONFLICTO DE INTERES",
    "descripcion": "ARTICULO 58. INCURRE EN ACTUACION BAJO CONFLICTO DE INTERES EL SERVIDOR PUBLICO QUE INTERVENGA POR MOTIVO DE SU EMPLEO, CARGO O COMISION EN CUALQUIER FORMA, EN LA ATENCION, TRAMITACION O RESOLUCION DE ASUNTOS EN LOS QUE TENGA CONFLICTO DE INTERES O IMPEDIMENTO LEGAL. AL TENER CONOCIMIENTO DE LOS ASUNTOS MENCIONADOS EN EL PARRAFO ANTERIOR, EL SERVIDOR PUBLICO INFORMARA TAL SITUACION AL JEFE INMEDIATO O AL ORGANO QUE DETERMINE LAS DISPOSICIONES APLICABLES DE LOS ENTES PUBLICOS, SOLICITANDO SEA EXCUSADO DE PARTICIPAR EN CUALQUIER FORMA EN LA ATENCION, TRAMITACION O RESOLUCION DE LOS MISMOS. SERA OBLIGACION DEL JEFE INMEDIATO DETERMINAR Y COMUNICARLE AL SERVIDOR PUBLICO, A MAS TARDAR 48 HORAS ANTES DEL PLAZO ESTABLECIDO PARA ATENDER EL ASUNTO EN CUESTION, LOS CASOS EN QUE NO SEA POSIBLE ABSTENERSE DE INTERVENIR EN LOS ASUNTOS, ASI COMO ESTABLECER INSTRUCCIONES POR ESCRITO PARA LA ATENCION, TRAMITACION O RESOLUCION IMPARCIAL Y OBJETIVA DE DICHOS ASUNTOS. CONFLICTO DE INTERES: LA POSIBLE AFECTACION DEL DESEMPEÑO IMPARCIAL Y OBJETIVO DE LAS FUNCIONES DE LOS SERVIDORES PUBLICOS EN RAZON DE INTERESES PERSONALES, FAMILIARES O DE NEGOCIOS;"
    },
    "tipoSancion": [
    {
        "clave": "I",
        "valor": "INHABILITADO"
    }
    ],
    "causaMotivoHechos": "EN FECHA 05 DE MAYO DE 2021, SE RECIBIO EN LA DIRECCION DE INVESTIGACION DE LA SECRETARIA DE LA CONTRALORIA DEL PODER EJECUTIVO DEL ESTADO DE QUERETARO, LA PROMOCION CIUDADANA 2021W2488 FORMULADA POR VANESSA GUTIERREZ MONROY, EN LA QUE DENUNCIO: QUE LA DENUNCIADA, MA. TERESA ADRIANA FLORES RUIZ, PRESIDENTE AUXILIAR DE LA JUNTA ESPECIAL UNO, TRAMITO EL CUMPLIMIENTO DE LA SENTENCIA DE AMPARO 788/2020 DEL JUZGADO PRIMERO DISTRITO DE PROCESOS PENALES FEDERALES Y DE AMPARO DE QUERETARO, EN LA QUE EL ABOGADO DE UNA DE LAS PARTES ES HERMANO DE LA PRESUNTA RESPONSABLE, EL LIC. NESTOR GUADALUPE FLORES RUIZ. 2.- EN FECHA 10 DE SEPTIEMBRE DE 2021, SE DICTO AUTO DE INICIO DE INVESTIGACION ADMINISTRATIVA NUMERO SC/SUB/DI/DDI-A/077/2021 EN EL QUE SE ADVIERTE QUE LA PRESUNTA RESPONSABLE AL DAR CUMPLIMIENTO A LA SENTENCIA DE AMPARO EN EL JUICIO LABORAL 0974/2012/1/1, EL ABOGADO PRINCIPAL DE LA PARTE ACTORA EN DICHO ASUNTO LO ES EL HERMANO DE LA FUNCIONARIA PUBLICA, POR LO QUE, AL TENER INTERES DIRECTO EN PRIMER TERMINO DEBIO EXCUSARSE DE CONOCER DEL ASUNTO Y NO LO HIZO, ADICIONAL, AL PRETENDER DAR CUMPLIMIENTO A LA EJECUTORIA, EMITE EL ACUERDO DE FECHA 20 DE ABRIL DEL PRESENTE AÑO, DONDE VENTAJOSAMENTE Y CON LA INTENCION DE AYUDAR A SU HERMANO QUE ASESORA A LA PARTE ACTORA EN EL PRESENTE JUICIO, ACLARA EL NOMBRE DE LA PERSONA A QUIEN SE DEMANDA Y EL DOMICILIO DONDE SE LE PUEDE LOCALIZAR PARA FAVORECER E IMPULSAR EL EXPEDIENTE, CUANDO A TODAS LUCES, EN LA DEMANDA INICIAL, NUNCA SE PROPORCIONARON ESOS DATOS. 2.- LA AUTORIDAD INVESTIGADORA EN FECHA 28 DE JUNIO DE 2022, EMITIO EL ACUERDO DE CALIFICACION DENTRO DEL CUADERNO DE INVESTIGACION NUMERO SC/SUB/DI/DDI-A/077/2021 CALIFICANDO LA FALTA COMO GRAVE (ACTUACION BAJO CONFLICTO DE INTERES). 3.- EN FECHA 28 DE OCTUBRE DE 2022, EL TITULAR DE LA DIRECCION DE INVESTIGACION DE LA SECRETARIA DE LA CONTRALORIA DEL PODER EJECUTIVO DEL ESTADO DE QUERETARO, EMITIO EL INFORME DE PRESUNTA RESPONSABILIDAD ADMINISTRATIVA.",
    "resolucion": {
    "url": "https://drive.google.com/file/d/1Dbzt7YOWvz9KgijPZWFJ2zQShAfFqAWV/view?usp=drive_link",
    "fechaResolucion": "2023-07-04"
    },
    "inhabilitacion": {
    "plazo": "UN AÑO A PARTIR DEL DIA SIGUIENTE EN QUE SE EFECTUE LA INSCRIPCION CORRESPONDIENTE EN LA PLATAFORMA DIGITAL NACIONAL DEL SISTEMA DE SERVIDORES PUBLICOS Y PARTICULARES SANCIONADOS.",
    "fechaInicial": "2023-09-08",
    "fechaFinal": "2024-09-07"
    },
    "observaciones": "EXISTEN ELEMENTOS DE PRUEBA QUE ACREDITAN QUE LA PRESUNTA RESPONSABLE NO SE EXCUSO DE CONOCER Y TRAMITAR EL JUICIO LABORAL 974/2012/1/1 Y EL INCIDENTE 788/202-II, YA QUE EN DICHO PROCEDIMIENTO EL ABOGADO DE LA PARTE ACTORA ES SU HERMANO, POR ELLO, QUEDO ACREDITADO QUE LA ENCAUSADA NO SE EXCUSO PARA ACTUAR EN EL JUICIO LABORAL 974/2012/1/1 Y EL INCIDENTE 788/202-II, DONDE EL ABOGADO DE LA PARTE ACTORA ES SU HERMANO Y POR ELLO ACTUO BAJO CONFLICTO DE INTERES, LO CUAL CONDUCE A DETERMINAR QUE MEDIANTE SU CONDUCTA CONTRAVINO EL PRINCIPIO DE DISCIPLINA DE IMPARCIALIDAD PREVISTO EN EL ARTICULO 7 FRACCION IX, DE LA LEY GENERAL DE RESPONSABILIDADES ADMINISTRATIVAS, POR LO CUAL SE HACE CREEDORA A LA SANCION DE INHABILITACION POR UN AÑO QUE CORRESPONDE AL MAXIMO DEL MARGEN CUANDO NO EXISTA BENEFICIO O LUCRO ALGUNO. SE TIENE EN CUENTA COMO HECHO NOTORIO QUE EN FECHA 11 DE MARZO DE 2022 FUE PUBLICADO EN EL PERIODICO OFICIAL DEL GOBIERNO DEL ESTADO DE QUERETARO, EL DICTAMEN DEFINITIVO POR EL QUE SE CONCEDE JUBILACION A LA C. MA. TERESA ADRIANA FLORES RUIZ, POR LO CUAL SE ACREDITA QUE LA ENCAUSADA NO SE ENCUENTRA DESEMPEÑANDO EL EMPLEO DE PRESIDENTA AUXILIAR, ADSCRITA A LA JUNTA LOCAL DE CONCILIACION Y ARBITRAJE DE LA SECRETARIA DEL TRABAJO DEL PODER EJECUTIVO DEL ESTADO DE QUERETARO."
}



In [77]:
result, campos_presentes, campos_faltantes = extract_faltas_element(json_data)

print("Datos extraídos:")
print(json.dumps(result, indent=2))
print("\nCampos presentes:", campos_presentes)
print("\nCampos faltantes:", campos_faltantes)

Datos extraídos:
{
  "fecha": "2023-09-21T00:00:00Z",
  "expediente": "2301002-SG-SE",
  "observaciones": "EXISTEN ELEMENTOS DE PRUEBA QUE ACREDITAN QUE LA PRESUNTA RESPONSABLE NO SE EXCUSO DE CONOCER Y TRAMITAR EL JUICIO LABORAL 974/2012/1/1 Y EL INCIDENTE 788/202-II, YA QUE EN DICHO PROCEDIMIENTO EL ABOGADO DE LA PARTE ACTORA ES SU HERMANO, POR ELLO, QUEDO ACREDITADO QUE LA ENCAUSADA NO SE EXCUSO PARA ACTUAR EN EL JUICIO LABORAL 974/2012/1/1 Y EL INCIDENTE 788/202-II, DONDE EL ABOGADO DE LA PARTE ACTORA ES SU HERMANO Y POR ELLO ACTUO BAJO CONFLICTO DE INTERES, LO CUAL CONDUCE A DETERMINAR QUE MEDIANTE SU CONDUCTA CONTRAVINO EL PRINCIPIO DE DISCIPLINA DE IMPARCIALIDAD PREVISTO EN EL ARTICULO 7 FRACCION IX, DE LA LEY GENERAL DE RESPONSABILIDADES ADMINISTRATIVAS, POR LO CUAL SE HACE CREEDORA A LA SANCION DE INHABILITACION POR UN A\u00d1O QUE CORRESPONDE AL MAXIMO DEL MARGEN CUANDO NO EXISTA BENEFICIO O LUCRO ALGUNO. SE TIENE EN CUENTA COMO HECHO NOTORIO QUE EN FECHA 11 DE MARZO DE 2022 

In [17]:
def extract_faltas_element(element_data):
  # Función auxiliar para acceder de forma segura a atributos anidados
  def safe_get(obj, *attrs):
    for attr in attrs:
      if not hasattr(obj, attr):
        return None
      obj = getattr(obj, attr)
    return obj

  # Lista para almacenar los campos faltantes
  campos_faltantes = []

  # Construir el diccionario de datos generales
  datos_generales = {
    'nombre': safe_get(element_data, 'servidorPublicoSancionado', 'nombres'),
    'primerApellido': safe_get(element_data, 'servidorPublicoSancionado', 'primerApellido'),
    'puesto': safe_get(element_data, 'servidorPublicoSancionado', 'puesto'),    
  }

  # Añadir genero, segundoApellido, curp o rfc solo si existe
  nivel = safe_get(element_data, 'servidorPublicoSancionado', 'nivel')
  if nivel is not None:
    datos_generales['nivel'] = nivel
  else:
    campos_faltantes.append('nivel')

  genero = safe_get(element_data, 'servidorPublicoSancionado', 'sexo')
  if genero is not None:
    datos_generales['sexo'] = genero
  else:
    campos_faltantes.append('sexo')

  segundoApellido = safe_get(element_data, 'servidorPublicoSancionado', 'segundoApellido')
  if segundoApellido is not None:
    datos_generales['segundoApellido'] = segundoApellido
  else:
    campos_faltantes.append('segundoApellido')

  curp = safe_get(element_data, 'servidorPublicoSancionado', 'curp')
  if curp is not None:
    datos_generales['curp'] = curp
  else:
    campos_faltantes.append('curp')

  rfc = safe_get(element_data, 'servidorPublicoSancionado', 'rfc')
  if rfc is not None:
    datos_generales['rfc'] = rfc
  else:
    campos_faltantes.append('rfc')
  
  # Construir el diccionario procesado
  processed_element = {
    'fecha': safe_get(element_data, 'fechaCaptura'),
    'expediente': safe_get(element_data, 'expediente'),
    'datosGenerales': datos_generales
  }

  return processed_element, campos_faltantes

In [38]:
def extract_missing_data(element_data: Dict[str, Any]) -> tuple[Dict[str, Any], list[str]]:
    """Extrae datos y detecta campos faltantes."""
    
    # Función auxiliar para acceder a atributos de forma segura
    def safe_get(obj: Any, attr: str, default: Optional[Any] = None) -> Any:
        return getattr(obj, attr, default)
    
    # Función auxiliar para agregar campos opcionales
    def add_optional_field(data: Dict[str, Any], obj: Any, attr: str, field_name: str) -> None:
        value = safe_get(obj, attr)
        if value is not None:
            data[field_name] = value
        else:
            campos_faltantes.append(field_name)
    
    campos_faltantes: list[str] = []
    
    # Construir diccionario de datos generales
    datos_generales = {
        'nombre': safe_get(element_data, 'servidorPublicoSancionado', 'nombres'),
        'primerApellido': safe_get(element_data, 'servidorPublicoSancionado', 'primerApellido'),
        'puesto': safe_get(element_data, 'servidorPublicoSancionado', 'puesto'),
    }
    
    # Agregar campos opcionales
    servidor_publico = element_data.get('servidorPublicoSancionado')
    if servidor_publico:
        add_optional_field(datos_generales, servidor_publico, 'nivel', 'nivel')
        add_optional_field(datos_generales, servidor_publico, 'sexo', 'sexo')
        add_optional_field(datos_generales, servidor_publico, 'segundoApellido', 'segundoApellido')
        add_optional_field(datos_generales, servidor_publico, 'curp', 'curp')
        add_optional_field(datos_generales, servidor_publico, 'rfc', 'rfc')
    
    # Construir diccionario procesado
    processed_element = {
        'fecha': safe_get(element_data, 'fechaCaptura'),
        'expediente': safe_get(element_data, 'expediente'),
        'datosGenerales': datos_generales
    }
    
    return processed_element, campos_faltantes

In [39]:
### Extraer datos de un solo elemento del array
element_data = data[0]
##pprint(element_data)
processed_element, campos_faltantes = extract_faltas_element(element_data)


In [40]:
pprint(processed_element)
pprint(campos_faltantes)

{'datosGenerales': {'curp': {},
                    'nivel': 'DATOS NO PROPORCIONADOS',
                    'nombre': 'ANDRES',
                    'primerApellido': 'REYES',
                    'puesto': 'FISCAL DEL MINISTERIO PUBLICO, ADSCRITO A LA '
                              'FISCALIA DE LA MUJER',
                    'rfc': {},
                    'segundoApellido': 'ENRIQUEZ',
                    'sexo': {}},
 'expediente': 'CA/012/2022',
 'fecha': '2022-05-06T00:00:00Z'}
[]


In [ ]:
pprint(type(data))
pprint(len(data))
pprint(len(data[0].keys()))
pprint(data[0].keys())
pprint(data[0])
pprint(data[0].id)
pprint(data[0].tipoFalta.valor)